# Automobile Dataset Description

#### This dataset consist of data From 1985 Ward's Automotive Yearbook.

#### This data set consists of three types of entities: 
 - (a) The specification of an auto in terms of various characteristics
 - (b) Its assigned insurance risk rating
 - (c) Its normalized losses in use as compared to other cars. 

#### The second rating corresponds to the degree to which the auto is more risky than its price indicates. Cars are initially assigned a risk factor symbol associated with its price. Then if it is more risky (or less), this symbol is adjusted by moving it up (or down) the scale. Actuarians call this process "symboling". A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.

#### The third factor is the relative averages loss payment per insured vehicle year. This value is normalized for all autos within a particular size classification (two-door small, station wagons, sports/speciality, etc...), and represents the averages loss per car per year.

## Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
#import seaborn as sns
# from matplotlib import pyplot as plt
# from matplotlib import style

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#from alibi.explainers import AnchorTabular
import joblib
#from retrieve_minio import *

#from numba import jit, cuda

from minio import Minio
#from minio.error import ResponseError

from subprocess import run, Popen, PIPE

In [2]:
MINIO_HOST="minio-service.kubeflow:9000"
MINIO_ACCESS_KEY="minio"
MINIO_SECRET_KEY="minio123"
MINIO_MODEL_BUCKET="seldon"
INCOME_MODEL_PATH="sklearn/automobile/model"
EXPLAINER_MODEL_PATH="sklearn/automobile/explainer"
OUTLIER_MODEL_PATH="sklearn/automobile/outlier"
DEPLOY_NAMESPACE="kubeflow"

## Deploy Model

In [3]:
secret = f"""apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
  namespace: {DEPLOY_NAMESPACE}
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: {MINIO_ACCESS_KEY}
  AWS_SECRET_ACCESS_KEY: {MINIO_SECRET_KEY}
  AWS_ENDPOINT_URL: http://{MINIO_HOST}
  USE_SSL: "false"
"""
with open("secret.yaml","w") as f:
    f.write(secret)
    
run("kubectl create namespace seldon", shell=True)
run("kubectl label namespace seldon istio-injection=enabled", shell=True)
run("cat secret.yaml | kubectl apply -f -", shell=True)

CompletedProcess(args='cat secret.yaml | kubectl apply -f -', returncode=127)

In [4]:
sa = f"""apiVersion: v1
kind: ServiceAccount
metadata:
  name: minio-sa
  namespace: {DEPLOY_NAMESPACE}
secrets:
  - name: seldon-init-container-secret
"""
with open("sa.yaml","w") as f:
    f.write(sa)
run("kubectl apply -f sa.yaml", shell=True)

CompletedProcess(args='kubectl apply -f sa.yaml', returncode=127)

In [5]:
model_yaml=f"""apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: automobile-classifier
  namespace: {DEPLOY_NAMESPACE}
spec:
  predictors:
  -  graph:
      implementation: SKLEARN_SERVER
      modelUri: s3://{MINIO_MODEL_BUCKET}/{INCOME_MODEL_PATH}
      envSecretRefName: seldon-init-container-secret
      name: classifier
      logger:
         mode: all
     name: default
     replicas: 1
"""
with open("model.yaml","w") as f:
    f.write(model_yaml)
run("kubectl apply -f model.yaml", shell=True)

CompletedProcess(args='kubectl apply -f model.yaml', returncode=127)

## Model Testing

In [6]:
payload='{"data": {"ndarray": [[0,188,2,1,0,2,3,2,0,101,176,64,54,2710,3,3,164,5,3,3,9,121,4250,21,28]]}}'
cmd=f"""curl -d '{payload}' \
   http://automobile-classifier-default.{DEPLOY_NAMESPACE}:8000/api/v1.0/predictions \
   -H "Content-Type: application/json"
"""
ret = Popen(cmd, shell=True,stdout=PIPE)
raw = ret.stdout.read().decode("utf-8")
print(raw)

{"data":{"names":["t:0","t:1","t:2","t:3","t:4","t:5","t:6","t:7","t:8","t:9","t:10","t:11","t:12","t:13","t:14","t:15","t:16","t:17","t:18","t:19","t:20","t:21","t:22","t:23","t:24","t:25","t:26","t:27","t:28","t:29","t:30","t:31","t:32","t:33","t:34","t:35","t:36","t:37","t:38","t:39","t:40","t:41","t:42","t:43","t:44","t:45","t:46","t:47","t:48","t:49","t:50","t:51","t:52","t:53","t:54","t:55","t:56","t:57","t:58","t:59","t:60","t:61","t:62","t:63","t:64","t:65","t:66","t:67","t:68","t:69","t:70","t:71","t:72","t:73","t:74","t:75","t:76","t:77","t:78","t:79","t:80","t:81","t:82","t:83","t:84","t:85","t:86","t:87","t:88","t:89","t:90","t:91","t:92","t:93","t:94","t:95","t:96","t:97","t:98","t:99","t:100","t:101","t:102","t:103","t:104","t:105","t:106","t:107","t:108","t:109","t:110","t:111","t:112","t:113","t:114","t:115","t:116","t:117","t:118","t:119","t:120","t:121","t:122","t:123","t:124","t:125","t:126","t:127","t:128","t:129","t:130","t:131","t:132","t:133"],"ndarray":[[0.0,0.0

In [7]:
payload='{"data": {"ndarray": [[0,188,2,1,0,2,3,2,0,101,176,64,54,2710,3,3,164,5,3,3,9,121,4250,21,28]]}}'
cmd=f"""curl -d '{payload}' \
   http://20.195.98.34/seldon/{DEPLOY_NAMESPACE}/automobile-classifier/api/v0.1/predictions \
   -H "Content-Type: application/json"
"""
ret = Popen(cmd, shell=True,stdout=PIPE)
raw = ret.stdout.read().decode("utf-8")
print(raw)

{"data":{"names":["t:0","t:1","t:2","t:3","t:4","t:5","t:6","t:7","t:8","t:9","t:10","t:11","t:12","t:13","t:14","t:15","t:16","t:17","t:18","t:19","t:20","t:21","t:22","t:23","t:24","t:25","t:26","t:27","t:28","t:29","t:30","t:31","t:32","t:33","t:34","t:35","t:36","t:37","t:38","t:39","t:40","t:41","t:42","t:43","t:44","t:45","t:46","t:47","t:48","t:49","t:50","t:51","t:52","t:53","t:54","t:55","t:56","t:57","t:58","t:59","t:60","t:61","t:62","t:63","t:64","t:65","t:66","t:67","t:68","t:69","t:70","t:71","t:72","t:73","t:74","t:75","t:76","t:77","t:78","t:79","t:80","t:81","t:82","t:83","t:84","t:85","t:86","t:87","t:88","t:89","t:90","t:91","t:92","t:93","t:94","t:95","t:96","t:97","t:98","t:99","t:100","t:101","t:102","t:103","t:104","t:105","t:106","t:107","t:108","t:109","t:110","t:111","t:112","t:113","t:114","t:115","t:116","t:117","t:118","t:119","t:120","t:121","t:122","t:123","t:124","t:125","t:126","t:127","t:128","t:129","t:130","t:131","t:132","t:133"],"ndarray":[[0.0,0.0